In [1]:
pip install pyagrum

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 38.0 MB/s eta 0:00:00


In [2]:
import pyAgrum as gum

In [3]:
Lanes=["Top","Jungle","Mid","ADC","Support"]

In [4]:
probabilitiesTop={"gold":[0.49651, 0.50349],"dmg":[0.49197, 0.50803],"Wr":[0.74075, 0.25925]}
probabilitiesJg={"gold":[0.5014, 0.4986],"dmg":[0.50314, 0.49686],"Wr":[0.74808, 0.25192]}
probabilitiesMid={"gold":[0.52128, 0.47872],"dmg":[0.52408, 0.47592],"Wr":[0.77634, 0.22366]}
probabilitiesAdc={"gold":[0.52268, 0.47732],"dmg":[0.51047, 0.48953],"Wr":[0.73552, 0.26448]}
probabilitiesSupp={"gold":[0.51256, 0.48744],"dmg":[0.50209, 0.49791],"Wr":[0.74703, 0.25297]}

probabilitiesLanes = {}

probabilitiesLanes["Top"] = probabilitiesTop
probabilitiesLanes["Jungle"] = probabilitiesJg
probabilitiesLanes["Mid"] = probabilitiesMid
probabilitiesLanes["ADC"] = probabilitiesAdc
probabilitiesLanes["Support"] = probabilitiesSupp


In [5]:
Net=gum.BayesNet()
print(Net)

BN{nodes: 0, arcs: 0, domainSize: 1, dim: 0, mem: 0o}


Agregando variables

## Creacion de la red bayesiana

In [6]:

def generateVariablesForLane(lane,bayesianNet):
  laneObj={}
  laneObj["gold"]=bayesianNet.add(gum.LabelizedVariable(f'G{lane}',f'Gold in {lane}',2))
  bayesianNet.cpt(laneObj["gold"]).fillWith(probabilitiesLanes[lane]['gold'])
  laneObj["dmg"]=bayesianNet.add(gum.LabelizedVariable(f'D{lane}',f'Damage in {lane}',2))
  bayesianNet.cpt(laneObj["dmg"]).fillWith(probabilitiesLanes[lane]['dmg'])
  laneObj["Wr"]=bayesianNet.add(gum.LabelizedVariable(f'WR{lane}',f'WR in {lane}',2))
  bayesianNet.cpt(laneObj["Wr"]).fillWith(probabilitiesLanes[lane]['Wr'])

  return laneObj
def addLaneArcs(laneObj,bayesianNet):
  bayesianNet.addArc(laneObj["gold"],teamWonGold)
  bayesianNet.addArc(laneObj["dmg"],teamWonDamage)
  bayesianNet.addArc(laneObj["Wr"],teamWonWinrate)


gw=Net.add(gum.LabelizedVariable(f'GameWon',f'GameWon',2))
teamWonGold = Net.add(gum.LabelizedVariable("TG","Team Gold",2))
teamWonDamage = Net.add(gum.LabelizedVariable("TD","Team Damage",2))
teamWonWinrate = Net.add(gum.LabelizedVariable("TW","Team Winrate",2))
Net.addArc(teamWonGold,gw)
Net.addArc(teamWonDamage,gw)
Net.addArc(teamWonWinrate,gw)

laneVariables = {}

for lane in Lanes:
  laneVariables[lane]=generateVariablesForLane(lane,Net)
  addLaneArcs(laneVariables[lane],Net)


In [7]:
print(Net)

BN{nodes: 19, arcs: 18, domainSize: 524288, dim: 119, mem: 1Ko 880o}


In [8]:
import pyAgrum.lib.notebook as gnb
gnb.getBN(Net,size="1000")

'<svg xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" width="1444pt" height="188pt" viewBox="0.00 0.00 1443.79 188.00">\n<g id="graph0" class="graph" transform="scale(1 1) rotate(0) translate(4 184)">\n<title>G</title>\n<!-- WRJungle -->\n<g id="node1" class="node">\n<title>WRJungle</title>\n<g id="a_node1"><a xlink:title="(9) WRJungle">\n<ellipse fill="#404040" stroke="black" cx="963.5" cy="-162" rx="48.19" ry="18"/>\n<text text-anchor="middle" x="963.5" y="-158.3" font-family="Times,serif" font-size="14.00" fill="white">WRJungle</text>\n</a>\n</g>\n</g>\n<!-- TW -->\n<g id="node11" class="node">\n<title>TW</title>\n<g id="a_node11"><a xlink:title="(3) TW">\n<ellipse fill="#404040" stroke="black" cx="1124.5" cy="-90" rx="27" ry="18"/>\n<text text-anchor="middle" x="1124.5" y="-86.3" font-family="Times,serif" font-size="14.00" fill="white">TW</text>\n</a>\n</g>\n</g>\n<!-- WRJungle&#45;&gt;TW -->\n<g id="edge2" class="edge">\n<title>WRJungle-&gt;TW</title>

In [9]:
Net.cpt(laneVariables['Top']['gold'])

(pyAgrum.Potential<double>@0x38e85f0) 
  GTop             |
0        |1        |
---------|---------|
 0.4965  | 0.5035  |

In [10]:
Net.cpt(teamWonGold)[{'GTop': 0, 'GJungle': 0, 'GMid': 0, 'GADC': 0, 'GSupport': 0}]= [1.0, 0]
Net.cpt(teamWonGold)[{'GTop': 0, 'GJungle': 0, 'GMid': 0, 'GADC': 0, 'GSupport': 1}]= [0.8, 0.2]
Net.cpt(teamWonGold)[{'GTop': 0, 'GJungle': 0, 'GMid': 0, 'GADC': 1, 'GSupport': 0}]= [0.8, 0.2]
Net.cpt(teamWonGold)[{'GTop': 0, 'GJungle': 0, 'GMid': 0, 'GADC': 1, 'GSupport': 1}]= [0.6, 0.4]
Net.cpt(teamWonGold)[{'GTop': 0, 'GJungle': 0, 'GMid': 1, 'GADC': 0, 'GSupport': 0}]= [0.8, 0.2]
Net.cpt(teamWonGold)[{'GTop': 0, 'GJungle': 0, 'GMid': 1, 'GADC': 0, 'GSupport': 1}]= [0.6, 0.4]
Net.cpt(teamWonGold)[{'GTop': 0, 'GJungle': 0, 'GMid': 1, 'GADC': 1, 'GSupport': 0}]= [0.6, 0.4]
Net.cpt(teamWonGold)[{'GTop': 0, 'GJungle': 0, 'GMid': 1, 'GADC': 1, 'GSupport': 1}]= [0.4, 0.6]
Net.cpt(teamWonGold)[{'GTop': 0, 'GJungle': 1, 'GMid': 0, 'GADC': 0, 'GSupport': 0}]= [0.8, 0.2]
Net.cpt(teamWonGold)[{'GTop': 0, 'GJungle': 1, 'GMid': 0, 'GADC': 0, 'GSupport': 1}]= [0.6, 0.4]
Net.cpt(teamWonGold)[{'GTop': 0, 'GJungle': 1, 'GMid': 0, 'GADC': 1, 'GSupport': 0}]= [0.6, 0.4]
Net.cpt(teamWonGold)[{'GTop': 0, 'GJungle': 1, 'GMid': 0, 'GADC': 1, 'GSupport': 1}]= [0.4, 0.6]
Net.cpt(teamWonGold)[{'GTop': 0, 'GJungle': 1, 'GMid': 1, 'GADC': 0, 'GSupport': 0}]= [0.6, 0.4]
Net.cpt(teamWonGold)[{'GTop': 0, 'GJungle': 1, 'GMid': 1, 'GADC': 0, 'GSupport': 1}]= [0.4, 0.6]
Net.cpt(teamWonGold)[{'GTop': 0, 'GJungle': 1, 'GMid': 1, 'GADC': 1, 'GSupport': 0}]= [0.4, 0.6]
Net.cpt(teamWonGold)[{'GTop': 0, 'GJungle': 1, 'GMid': 1, 'GADC': 1, 'GSupport': 1}]= [0.2, 0.8]
Net.cpt(teamWonGold)[{'GTop': 1, 'GJungle': 0, 'GMid': 0, 'GADC': 0, 'GSupport': 0}]= [0.8, 0.2]
Net.cpt(teamWonGold)[{'GTop': 1, 'GJungle': 0, 'GMid': 0, 'GADC': 0, 'GSupport': 1}]= [0.6, 0.4]
Net.cpt(teamWonGold)[{'GTop': 1, 'GJungle': 0, 'GMid': 0, 'GADC': 1, 'GSupport': 0}]= [0.6, 0.4]
Net.cpt(teamWonGold)[{'GTop': 1, 'GJungle': 0, 'GMid': 0, 'GADC': 1, 'GSupport': 1}]= [0.4, 0.6]
Net.cpt(teamWonGold)[{'GTop': 1, 'GJungle': 0, 'GMid': 1, 'GADC': 0, 'GSupport': 0}]= [0.6, 0.4]
Net.cpt(teamWonGold)[{'GTop': 1, 'GJungle': 0, 'GMid': 1, 'GADC': 0, 'GSupport': 1}]= [0.4, 0.6]
Net.cpt(teamWonGold)[{'GTop': 1, 'GJungle': 0, 'GMid': 1, 'GADC': 1, 'GSupport': 0}]= [0.4, 0.6]
Net.cpt(teamWonGold)[{'GTop': 1, 'GJungle': 0, 'GMid': 1, 'GADC': 1, 'GSupport': 1}]= [0.2, 0.8]
Net.cpt(teamWonGold)[{'GTop': 1, 'GJungle': 1, 'GMid': 0, 'GADC': 0, 'GSupport': 0}]= [0.6, 0.4]
Net.cpt(teamWonGold)[{'GTop': 1, 'GJungle': 1, 'GMid': 0, 'GADC': 0, 'GSupport': 1}]= [0.4, 0.6]
Net.cpt(teamWonGold)[{'GTop': 1, 'GJungle': 1, 'GMid': 0, 'GADC': 1, 'GSupport': 0}]= [0.4, 0.6]
Net.cpt(teamWonGold)[{'GTop': 1, 'GJungle': 1, 'GMid': 0, 'GADC': 1, 'GSupport': 1}]= [0.2, 0.8]
Net.cpt(teamWonGold)[{'GTop': 1, 'GJungle': 1, 'GMid': 1, 'GADC': 0, 'GSupport': 0}]= [0.4, 0.6]
Net.cpt(teamWonGold)[{'GTop': 1, 'GJungle': 1, 'GMid': 1, 'GADC': 0, 'GSupport': 1}]= [0.2, 0.8]
Net.cpt(teamWonGold)[{'GTop': 1, 'GJungle': 1, 'GMid': 1, 'GADC': 1, 'GSupport': 0}]= [0.2, 0.8]
Net.cpt(teamWonGold)[{'GTop': 1, 'GJungle': 1, 'GMid': 1, 'GADC': 1, 'GSupport': 1}]= [0, 1.0]

In [11]:
Net.cpt(teamWonGold)

(pyAgrum.Potential<double>@0x35c6d00) 
                                  ||  TG               |
GTop  |GJungl|GMid  |GADC  |GSuppo||0        |1        |
------|------|------|------|------||---------|---------|
0     |0     |0     |0     |0     || 1.0000  | 0.0000  |
1     |0     |0     |0     |0     || 0.8000  | 0.2000  |
0     |1     |0     |0     |0     || 0.8000  | 0.2000  |
1     |1     |0     |0     |0     || 0.6000  | 0.4000  |
0     |0     |1     |0     |0     || 0.8000  | 0.2000  |
1     |0     |1     |0     |0     || 0.6000  | 0.4000  |
[...20 more line(s) ...]
0     |1     |0     |1     |1     || 0.4000  | 0.6000  |
1     |1     |0     |1     |1     || 0.2000  | 0.8000  |
0     |0     |1     |1     |1     || 0.4000  | 0.6000  |
1     |0     |1     |1     |1     || 0.2000  | 0.8000  |
0     |1     |1     |1     |1     || 0.2000  | 0.8000  |
1     |1     |1     |1     |1     || 0.0000  | 1.0000  |

In [12]:
Net.cpt(teamWonDamage)[{'DTop': 0, 'DJungle': 0, 'DMid': 0, 'DADC': 0, 'DSupport': 1}]= [0.9, 0.1]
Net.cpt(teamWonDamage)[{'DTop': 0, 'DJungle': 0, 'DMid': 0, 'DADC': 1, 'DSupport': 0}]= [0.7, 0.3]
Net.cpt(teamWonDamage)[{'DTop': 0, 'DJungle': 0, 'DMid': 0, 'DADC': 1, 'DSupport': 1}]= [0.6, 0.4]
Net.cpt(teamWonDamage)[{'DTop': 0, 'DJungle': 0, 'DMid': 1, 'DADC': 0, 'DSupport': 0}]= [0.7, 0.3]
Net.cpt(teamWonDamage)[{'DTop': 0, 'DJungle': 0, 'DMid': 1, 'DADC': 0, 'DSupport': 1}]= [0.6, 0.4]
Net.cpt(teamWonDamage)[{'DTop': 0, 'DJungle': 0, 'DMid': 1, 'DADC': 1, 'DSupport': 0}]= [0.4, 0.6]
Net.cpt(teamWonDamage)[{'DTop': 0, 'DJungle': 0, 'DMid': 1, 'DADC': 1, 'DSupport': 1}]= [0.3, 0.7]
Net.cpt(teamWonDamage)[{'DTop': 0, 'DJungle': 1, 'DMid': 0, 'DADC': 0, 'DSupport': 0}]= [0.9, 0.1]
Net.cpt(teamWonDamage)[{'DTop': 0, 'DJungle': 1, 'DMid': 0, 'DADC': 0, 'DSupport': 1}]= [0.8, 0.2]
Net.cpt(teamWonDamage)[{'DTop': 0, 'DJungle': 1, 'DMid': 0, 'DADC': 1, 'DSupport': 0}]= [0.6, 0.4]
Net.cpt(teamWonDamage)[{'DTop': 0, 'DJungle': 1, 'DMid': 0, 'DADC': 1, 'DSupport': 1}]= [0.5, 0.5]
Net.cpt(teamWonDamage)[{'DTop': 0, 'DJungle': 1, 'DMid': 1, 'DADC': 0, 'DSupport': 0}]= [0.6, 0.4]
Net.cpt(teamWonDamage)[{'DTop': 0, 'DJungle': 1, 'DMid': 1, 'DADC': 0, 'DSupport': 1}]= [0.5, 0.5]
Net.cpt(teamWonDamage)[{'DTop': 0, 'DJungle': 1, 'DMid': 1, 'DADC': 1, 'DSupport': 0}]= [0.3, 0.7]
Net.cpt(teamWonDamage)[{'DTop': 0, 'DJungle': 1, 'DMid': 1, 'DADC': 1, 'DSupport': 1}]= [0.2, 0.8]
Net.cpt(teamWonDamage)[{'DTop': 1, 'DJungle': 0, 'DMid': 0, 'DADC': 0, 'DSupport': 0}]= [0.8, 0.2]
Net.cpt(teamWonDamage)[{'DTop': 1, 'DJungle': 0, 'DMid': 0, 'DADC': 0, 'DSupport': 1}]= [0.7, 0.3]
Net.cpt(teamWonDamage)[{'DTop': 1, 'DJungle': 0, 'DMid': 0, 'DADC': 1, 'DSupport': 0}]= [0.5, 0.5]
Net.cpt(teamWonDamage)[{'DTop': 1, 'DJungle': 0, 'DMid': 0, 'DADC': 1, 'DSupport': 1}]= [0.4, 0.6]
Net.cpt(teamWonDamage)[{'DTop': 1, 'DJungle': 0, 'DMid': 1, 'DADC': 0, 'DSupport': 0}]= [0.5, 0.5]
Net.cpt(teamWonDamage)[{'DTop': 1, 'DJungle': 0, 'DMid': 1, 'DADC': 0, 'DSupport': 1}]= [0.4, 0.6]
Net.cpt(teamWonDamage)[{'DTop': 1, 'DJungle': 0, 'DMid': 1, 'DADC': 1, 'DSupport': 0}]= [0.2, 0.8]
Net.cpt(teamWonDamage)[{'DTop': 1, 'DJungle': 0, 'DMid': 1, 'DADC': 1, 'DSupport': 1}]= [0.1, 0.9]
Net.cpt(teamWonDamage)[{'DTop': 1, 'DJungle': 1, 'DMid': 0, 'DADC': 0, 'DSupport': 0}]= [0.7, 0.3]
Net.cpt(teamWonDamage)[{'DTop': 1, 'DJungle': 1, 'DMid': 0, 'DADC': 0, 'DSupport': 1}]= [0.6, 0.4]
Net.cpt(teamWonDamage)[{'DTop': 1, 'DJungle': 1, 'DMid': 0, 'DADC': 1, 'DSupport': 0}]= [0.4, 0.6]
Net.cpt(teamWonDamage)[{'DTop': 1, 'DJungle': 1, 'DMid': 0, 'DADC': 1, 'DSupport': 1}]= [0.3, 0.7]
Net.cpt(teamWonDamage)[{'DTop': 1, 'DJungle': 1, 'DMid': 1, 'DADC': 0, 'DSupport': 0}]= [0.4, 0.6]
Net.cpt(teamWonDamage)[{'DTop': 1, 'DJungle': 1, 'DMid': 1, 'DADC': 0, 'DSupport': 1}]= [0.3, 0.7]
Net.cpt(teamWonDamage)[{'DTop': 1, 'DJungle': 1, 'DMid': 1, 'DADC': 1, 'DSupport': 0}]= [0.1, 0.9]
Net.cpt(teamWonDamage)[{'DTop': 1, 'DJungle': 1, 'DMid': 1, 'DADC': 1, 'DSupport': 1}]= [0, 1.0]
Net.cpt(teamWonDamage)[{'DTop': 0, 'DJungle': 0, 'DMid': 0, 'DADC': 0, 'DSupport': 0}]= [1.0, 0.0]

In [13]:
Net.cpt(teamWonDamage)

(pyAgrum.Potential<double>@0x352f420) 
                                  ||  TD               |
DTop  |DJungl|DMid  |DADC  |DSuppo||0        |1        |
------|------|------|------|------||---------|---------|
0     |0     |0     |0     |0     || 1.0000  | 0.0000  |
1     |0     |0     |0     |0     || 0.8000  | 0.2000  |
0     |1     |0     |0     |0     || 0.9000  | 0.1000  |
1     |1     |0     |0     |0     || 0.7000  | 0.3000  |
0     |0     |1     |0     |0     || 0.7000  | 0.3000  |
1     |0     |1     |0     |0     || 0.5000  | 0.5000  |
[...20 more line(s) ...]
0     |1     |0     |1     |1     || 0.5000  | 0.5000  |
1     |1     |0     |1     |1     || 0.3000  | 0.7000  |
0     |0     |1     |1     |1     || 0.3000  | 0.7000  |
1     |0     |1     |1     |1     || 0.1000  | 0.9000  |
0     |1     |1     |1     |1     || 0.2000  | 0.8000  |
1     |1     |1     |1     |1     || 0.0000  | 1.0000  |

In [14]:
Net.cpt(teamWonWinrate)[{'WRTop': 0, 'WRJungle': 0, 'WRMid': 0, 'WRADC': 0, 'WRSupport': 0}]= [1.0, 0]
Net.cpt(teamWonWinrate)[{'WRTop': 0, 'WRJungle': 0, 'WRMid': 0, 'WRADC': 0, 'WRSupport': 1}]= [0.8, 0.2]
Net.cpt(teamWonWinrate)[{'WRTop': 0, 'WRJungle': 0, 'WRMid': 0, 'WRADC': 1, 'WRSupport': 0}]= [0.8, 0.2]
Net.cpt(teamWonWinrate)[{'WRTop': 0, 'WRJungle': 0, 'WRMid': 0, 'WRADC': 1, 'WRSupport': 1}]= [0.6, 0.4]
Net.cpt(teamWonWinrate)[{'WRTop': 0, 'WRJungle': 0, 'WRMid': 1, 'WRADC': 0, 'WRSupport': 0}]= [0.8, 0.2]
Net.cpt(teamWonWinrate)[{'WRTop': 0, 'WRJungle': 0, 'WRMid': 1, 'WRADC': 0, 'WRSupport': 1}]= [0.6, 0.4]
Net.cpt(teamWonWinrate)[{'WRTop': 0, 'WRJungle': 0, 'WRMid': 1, 'WRADC': 1, 'WRSupport': 0}]= [0.6, 0.4]
Net.cpt(teamWonWinrate)[{'WRTop': 0, 'WRJungle': 0, 'WRMid': 1, 'WRADC': 1, 'WRSupport': 1}]= [0.4, 0.6]
Net.cpt(teamWonWinrate)[{'WRTop': 0, 'WRJungle': 1, 'WRMid': 0, 'WRADC': 0, 'WRSupport': 0}]= [0.8, 0.2]
Net.cpt(teamWonWinrate)[{'WRTop': 0, 'WRJungle': 1, 'WRMid': 0, 'WRADC': 0, 'WRSupport': 1}]= [0.6, 0.4]
Net.cpt(teamWonWinrate)[{'WRTop': 0, 'WRJungle': 1, 'WRMid': 0, 'WRADC': 1, 'WRSupport': 0}]= [0.6, 0.4]
Net.cpt(teamWonWinrate)[{'WRTop': 0, 'WRJungle': 1, 'WRMid': 0, 'WRADC': 1, 'WRSupport': 1}]= [0.4, 0.6]
Net.cpt(teamWonWinrate)[{'WRTop': 0, 'WRJungle': 1, 'WRMid': 1, 'WRADC': 0, 'WRSupport': 0}]= [0.6, 0.4]
Net.cpt(teamWonWinrate)[{'WRTop': 0, 'WRJungle': 1, 'WRMid': 1, 'WRADC': 0, 'WRSupport': 1}]= [0.4, 0.6]
Net.cpt(teamWonWinrate)[{'WRTop': 0, 'WRJungle': 1, 'WRMid': 1, 'WRADC': 1, 'WRSupport': 0}]= [0.4, 0.6]
Net.cpt(teamWonWinrate)[{'WRTop': 0, 'WRJungle': 1, 'WRMid': 1, 'WRADC': 1, 'WRSupport': 1}]= [0.2, 0.8]
Net.cpt(teamWonWinrate)[{'WRTop': 1, 'WRJungle': 0, 'WRMid': 0, 'WRADC': 0, 'WRSupport': 0}]= [0.8, 0.2]
Net.cpt(teamWonWinrate)[{'WRTop': 1, 'WRJungle': 0, 'WRMid': 0, 'WRADC': 0, 'WRSupport': 1}]= [0.6, 0.4]
Net.cpt(teamWonWinrate)[{'WRTop': 1, 'WRJungle': 0, 'WRMid': 0, 'WRADC': 1, 'WRSupport': 0}]= [0.6, 0.4]
Net.cpt(teamWonWinrate)[{'WRTop': 1, 'WRJungle': 0, 'WRMid': 0, 'WRADC': 1, 'WRSupport': 1}]= [0.4, 0.6]
Net.cpt(teamWonWinrate)[{'WRTop': 1, 'WRJungle': 0, 'WRMid': 1, 'WRADC': 0, 'WRSupport': 0}]= [0.6, 0.4]
Net.cpt(teamWonWinrate)[{'WRTop': 1, 'WRJungle': 0, 'WRMid': 1, 'WRADC': 0, 'WRSupport': 1}]= [0.4, 0.6]
Net.cpt(teamWonWinrate)[{'WRTop': 1, 'WRJungle': 0, 'WRMid': 1, 'WRADC': 1, 'WRSupport': 0}]= [0.4, 0.6]
Net.cpt(teamWonWinrate)[{'WRTop': 1, 'WRJungle': 0, 'WRMid': 1, 'WRADC': 1, 'WRSupport': 1}]= [0.2, 0.8]
Net.cpt(teamWonWinrate)[{'WRTop': 1, 'WRJungle': 1, 'WRMid': 0, 'WRADC': 0, 'WRSupport': 0}]= [0.6, 0.4]
Net.cpt(teamWonWinrate)[{'WRTop': 1, 'WRJungle': 1, 'WRMid': 0, 'WRADC': 0, 'WRSupport': 1}]= [0.4, 0.6]
Net.cpt(teamWonWinrate)[{'WRTop': 1, 'WRJungle': 1, 'WRMid': 0, 'WRADC': 1, 'WRSupport': 0}]= [0.4, 0.6]
Net.cpt(teamWonWinrate)[{'WRTop': 1, 'WRJungle': 1, 'WRMid': 0, 'WRADC': 1, 'WRSupport': 1}]= [0.2, 0.8]
Net.cpt(teamWonWinrate)[{'WRTop': 1, 'WRJungle': 1, 'WRMid': 1, 'WRADC': 0, 'WRSupport': 0}]= [0.4, 0.6]
Net.cpt(teamWonWinrate)[{'WRTop': 1, 'WRJungle': 1, 'WRMid': 1, 'WRADC': 0, 'WRSupport': 1}]= [0.2, 0.8]
Net.cpt(teamWonWinrate)[{'WRTop': 1, 'WRJungle': 1, 'WRMid': 1, 'WRADC': 1, 'WRSupport': 0}]= [0.2, 0.8]
Net.cpt(teamWonWinrate)[{'WRTop': 1, 'WRJungle': 1, 'WRMid': 1, 'WRADC': 1, 'WRSupport': 1}]= [0, 1.0]

In [15]:
Net.cpt(teamWonWinrate)

(pyAgrum.Potential<double>@0x3455a50) 
                                  ||  TW               |
WRTop |WRJung|WRMid |WRADC |WRSupp||0        |1        |
------|------|------|------|------||---------|---------|
0     |0     |0     |0     |0     || 1.0000  | 0.0000  |
1     |0     |0     |0     |0     || 0.8000  | 0.2000  |
0     |1     |0     |0     |0     || 0.8000  | 0.2000  |
1     |1     |0     |0     |0     || 0.6000  | 0.4000  |
0     |0     |1     |0     |0     || 0.8000  | 0.2000  |
1     |0     |1     |0     |0     || 0.6000  | 0.4000  |
[...20 more line(s) ...]
0     |1     |0     |1     |1     || 0.4000  | 0.6000  |
1     |1     |0     |1     |1     || 0.2000  | 0.8000  |
0     |0     |1     |1     |1     || 0.4000  | 0.6000  |
1     |0     |1     |1     |1     || 0.2000  | 0.8000  |
0     |1     |1     |1     |1     || 0.2000  | 0.8000  |
1     |1     |1     |1     |1     || 0.0000  | 1.0000  |

In [16]:
Net.cpt(gw)

(pyAgrum.Potential<double>@0x35b0d00) 
                    ||  GameWon          |
TG    |TD    |TW    ||0        |1        |
------|------|------||---------|---------|
0     |0     |0     || 0.0000  | 0.0000  |
1     |0     |0     || 0.0000  | 0.0000  |
0     |1     |0     || 0.0000  | 0.0000  |
1     |1     |0     || 0.0000  | 0.0000  |
0     |0     |1     || 0.0000  | 0.0000  |
1     |0     |1     || 0.0000  | 0.0000  |
0     |1     |1     || 0.0000  | 0.0000  |
1     |1     |1     || 0.0000  | 0.0000  |

In [17]:
Net.cpt(gw)[{'TG': 0, 'TD': 0, 'TW': 0}]= [0.99588, 0.00412]
Net.cpt(gw)[{'TG': 0, 'TD': 0, 'TW': 1}]= [0.97978, 0.02022]
Net.cpt(gw)[{'TG': 0, 'TD': 1, 'TW': 0}]= [0.95935, 0.04065]
Net.cpt(gw)[{'TG': 0, 'TD': 1, 'TW': 1}]= [0.86443, 0.13557]
Net.cpt(gw)[{'TG': 1, 'TD': 0, 'TW': 0}]= [0.0886, 0.91140]
Net.cpt(gw)[{'TG': 1, 'TD': 0, 'TW': 1}]= [0.06495, 0.93505]
Net.cpt(gw)[{'TG': 1, 'TD': 1, 'TW': 0}]= [0.02691, 0.97309]
Net.cpt(gw)[{'TG': 1, 'TD': 1, 'TW': 1}]= [0.01033, 0.98967]

In [18]:
Net.cpt(gw)

(pyAgrum.Potential<double>@0x35b0d00) 
                    ||  GameWon          |
TG    |TD    |TW    ||0        |1        |
------|------|------||---------|---------|
0     |0     |0     || 0.9959  | 0.0041  |
1     |0     |0     || 0.0886  | 0.9114  |
0     |1     |0     || 0.9594  | 0.0406  |
1     |1     |0     || 0.0269  | 0.9731  |
0     |0     |1     || 0.9798  | 0.0202  |
1     |0     |1     || 0.0649  | 0.9351  |
0     |1     |1     || 0.8644  | 0.1356  |
1     |1     |1     || 0.0103  | 0.9897  |

In [19]:
ie=gum.LazyPropagation(Net)

#Sin ninguna evidencia vemos que el lado rojo=0 tiende a ganar un poco mas que el lado azul=1

In [20]:
ie.makeInference()
print(ie.posterior(gw))


  GameWon          |
0        |1        |
---------|---------|
 0.5187  | 0.4813  |



Imaginemos que nuestro Top viene ganando todo y con un campeon que le permite hacer mucho dinero, GP por ejemplo; Luego nuestro suppor lleva perdiento todas y nuestro mid tiene un campeon de mucho daño, entonces en base a esa evidencia tratemos de inferir quien gana

In [21]:
ie.setEvidence({'GTop':1,'WRSupport':0,'DMid':1})
print(ie.posterior(gw))


  GameWon          |
0        |1        |
---------|---------|
 0.4210  | 0.5790  |

